# 1: Preparing data

In [1]:
import pandas as pd
import numpy as np
import glob
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed, Masking
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load all CSV files
csv_files = glob.glob('./csv_ave/*.csv')  

X, Y = [], []  
df_all = pd.read_csv("all_c6_new.csv")  

n = 0
for file in csv_files:
    df = pd.read_csv(file)  

    # Extract metadata
    initial_speed = df_all["v_set\n[km/h]"][n]
    initial_pressure = df_all["Fb_set\n[kN]"][n]
    braking_time = df_all["braking_time"][n]  # Braking time is the number of rows in the file

    # Extract temperature sequence
    avg_temp_sequence = df["ave"].values  

    # Normalize inputs
    initial_speed /= 100  
    initial_pressure /= 23  
    braking_time /= max(df_all["braking_time"])  # Normalize braking time

    # Store inputs
    X.append([initial_speed, initial_pressure, braking_time])  # Now 3 features
    Y.append(avg_temp_sequence)  

    n += 1

# **Pad sequences to the maximum length**
max_seq_length = max(len(seq) for seq in Y)
Y_padded = pad_sequences(Y, maxlen=max_seq_length, dtype='float32', padding='post', value=0.0)

# Convert X to numpy array
X = np.array(X)  # Shape: (num_samples, 3)

# Reshape X for LSTM
X = X.reshape((X.shape[0], 1, X.shape[1]))  # Shape: (num_samples, 1, 3)

# Repeat X along the time dimension to match Y_padded
X = np.repeat(X, max_seq_length, axis=1)  # Shape: (num_samples, max_seq_length, 3)

# Reshape Y for LSTM
Y_padded = Y_padded.reshape((Y_padded.shape[0], Y_padded.shape[1], 1))  # Shape: (num_samples, max_seq_length, 1)

### **Define LSTM Model**


In [43]:
## save x
X1 = X.reshape(X.shape[0], -1)
pd_x = pd.DataFrame(X1)
pd_x.to_csv('x.csv')

12


In [63]:
# make x5 as X in colab
x2 = pd.read_csv('x.csv')  #datafrmae
x3 = x2.values #array
x4 = np.delete(x3, 0, axis = 1)
x5 = x4.reshape(12, 43664, 3)
x5.shape

(12, 43664, 3)

In [61]:
# validation
print(X.shape)
print(x5.shape)
print(np.allclose(X, x5)) ## X and X5 is nearly the same, sice it contains float, so should not use np.all(a == b)

(12, 43664, 3)
(12, 43664, 3)
True


In [67]:
# salve y
y1 = Y_padded.reshape(Y_padded.shape[0], -1)
pd_y = pd.DataFrame(y1)
pd_y.to_csv('y.csv')

In [70]:
# get the y5 as Y_padded in colab
y2 = pd.read_csv('y.csv')  #datafrmae
y3 = y2.values #array
y4 = np.delete(y3, 0, axis = 1)
y5 = y4.reshape(12, 43664, 1)
y5.shape

(12, 43664, 1)

# 2: Training model

In [ ]:

# Define the model
model = Sequential([
    Input(shape=(max_seq_length, 3)),  #  Define input explicitly
    Masking(mask_value=0.0),  # No need for input_shape here
    LSTM(64, return_sequences=True),
    LSTM(32, return_sequences=True),
    TimeDistributed(Dense(1))  # Output sequence shape (batch, time_steps, 1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Print model summary
model.summary()

# Train the model
model.fit(X, Y_padded, epochs=5, batch_size=8, validation_split=0.2)


# For same length

In [ ]:
import pandas as pd
import numpy as np
import glob
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed

csv_files = glob.glob('./csv_ave/*.csv')  # Assuming CSVs are stored in 'data/' folder

X, Y = [], []  # Lists to store input and output data
df_all = pd.read_csv("all_c6.csv")
n = 0

for file in csv_files:
    df = pd.read_csv(file)  # Load temperature data file
    
    # Extract metadata from df_all
    initial_speed = df_all["v_set\n[km/h]"][n]
    initial_pressure = df_all["Fb_set\n[kN]"][n]
    
    # Extract temperature sequence (all rows)
    avg_temp_sequence = df["ave"] # (time_steps,)

    # Normalize inputs
    initial_speed /= 100  # Assuming max speed ~100 km/h
    initial_pressure /= 23  # Assuming max pressure ~23 kN

    # Store in lists
    X.append([initial_speed, initial_pressure])  # Shape: (2,)
    Y.append(avg_temp_sequence)  # Shape: (time_steps,)

    n = n+ 1
    #print(n)

# Convert to numpy arrays
X = np.array(X)  # Shape: (num_tests, 2)
Y = np.array(Y)  # Shape: (num_tests, time_steps)

# Reshape X for LSTM (samples, timesteps, features)
X = X.reshape((X.shape[0], 1, X.shape[1]))  # Shape: (num_tests, 1, 2)

# Reshape Y for LSTM (samples, timesteps, features)
Y = Y.reshape((Y.shape[0], Y.shape[1], 1))  # Shape: (num_tests, time_steps, 1)


In [ ]:
# Define LSTM model
model = Sequential([
    Dense(16, activation="relu", input_shape=(1, 2)),  # Input: speed, pressure
    RepeatVector(Y.shape[1]),  # Repeat input vector for each time step
    LSTM(64, return_sequences=True),  # LSTM learns temperature pattern
    TimeDistributed(Dense(32, activation="relu")),  # Dense applied at each time step
    TimeDistributed(Dense(1))  # Output: temperature sequence
])

# Compile model
model.compile(optimizer="adam", loss="mse")

# Train model
model.fit(X, Y, epochs=20, batch_size=4, validation_split=0.2)

# Save model for future predictions
model.save("brake_temp_lstm.h5")


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load trained model
model = load_model("brake_temp_lstm.h5")

# Example input (Speed = 120 km/h, Pressure = 60 bar)
new_input = np.array([[120 / 200, 60 / 100]])  # Normalize values
new_input = new_input.reshape((1, 1, 2))  # Reshape for LSTM

# Predict temperature sequence
predicted_temp_sequence = model.predict(new_input)

# Print results
print(predicted_temp_sequence[0].flatten())  # Output: Temperature over time


In [ ]:
# 1: Print length of Temperature

In [ ]:
import pandas as pd
import numpy as np
import glob
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed

csv_files = glob.glob('./csv_ave/*.csv')  # Assuming CSVs are stored in 'data/' folder

X, Y = [], []  # Lists to store input and output data
df_all = pd.read_csv("all_c6_new.csv")
n = 0

for file in csv_files:
    df = pd.read_csv(file)  # Load temperature data file
    
    # Extract temperature sequence (all rows)
    avg_temp_sequence = df["ave"] # (time_steps,)


    Y.append(avg_temp_sequence.size)  # Shape: (time_steps,)


print(Y)